# Test Book
We will show the differences between different matching as well as secondary matching algorithms in multiple scenarios.
## Directory
1. Test, hungarian VS forloops
2. Test, only distances VS second match with mask iou >0
3. Test, mask_iou > 0 and distance VS distance+(1-mask_iou) in second match method.
4. Test, bbox_iou VS mask_iou
5. Conclusions

## Define the match functions for forloops/hungarian

In [ ]:
from scipy.optimize import linear_sum_assignment
import numpy as np
def forloops_match(distances, dis_thr):
    to_mark_matched = distances.copy()
    num_gt = distances.shape[0]
    num_pred = distances.shape[1]
    for i in range(num_gt):
        for j in range(num_pred):
            if distances[i, j] < dis_thr:
                print(f"TP: index is [{i}, {j}], row_index={i}, col_index={j}")
                to_mark_matched[i,j] = np.nan
                break 
    to_mark_matched= np.isnan(to_mark_matched)
    for i in range(num_gt):
        for j in range(num_pred):
            if not to_mark_matched[i,j]:
                print(f"FP: index is [{i}, {j}], row_index={i}, col_index={j}.")


def hungarian_match(distances, dis_thr):
    to_mark_matched = distances.copy()
    num_gt = distances.shape[0]
    num_pred = distances.shape[1]
    if np.all(np.isinf(distances)):
        # fix cost matrix feasible, like [[np.inf, np.inf]]
        row = np.empty(1)
        col = np.empty(1)
        matched = np.empty(0).astype("bool")
    else:
        row, col = linear_sum_assignment(distances)
        select_distances = distances[row, col]
        matched = select_distances < dis_thr
    for i, j in zip(row[matched], col[matched]):
        print(f"TP: index is [{i}, {j}], row_index={i}, col_index={j}")
        to_mark_matched[i,j] = np.nan
    to_mark_matched= np.isnan(to_mark_matched)
    for i in range(num_gt):
        for j in range(num_pred):
            if not to_mark_matched[i,j]:
                print(f"FP: index is [{i}, {j}], row_index={i}, col_index={j}.")

## Test, hungarian VS forloops

In this scenario, we assume that there is 1 GT, 3 Pred, row means GT and column means Pred.  
We assume that the Euclidean distance `dis_thr=3`.  
Obviously, we expect the result that the Pred with `index=[0, 1]` is assigned to that GT, since the Euclidean distance between the two is `distance=1`.  

In [ ]:
# Settings
distances = np.array([[2.5, 1, 1.8]])
dis_thr = 3
num_gt = distances.shape[0] # 1
num_pred = distances.shape[1] # 3

### Hungarian

In [ ]:
hungarian_match(distances, dis_thr)

### Forloops

In [ ]:
forloops_match(distances, dis_thr)

### Conclusion
Obviously, the Hungarian is more accurate!

## Test, only distances VS second match with mask iou >0


In this scenario, we assume that GT and the two Pred have the same Euclidean distance, but one `iou=0` and one `iou=0.8`.  
We assume that the Euclidean distance `dis_thr=3`.  
Obviously, we expect the Pred with `index=[0, 1]` to be assigned to this gt, because they have the `iou=0.8`.

In [ ]:
distances = np.array([[1.2, 1.2, 1.8]])
mask_iou = np.array([[0, 0.8, 0.]])
dis_thr = 3

### Forloops for pure distance matching

In [ ]:
forloops_match(distances, dis_thr)

### Hungarian for pure distance matching

In [ ]:
hungarian_match(distances, dis_thr)

### Forloops for second match method with mask_iou > 0

In [ ]:
# first, filter out the non-overlapping pairs of gt and pred. 
# This is much faster than "first match -> second match".
tmp_mask_iou = mask_iou.copy()
tmp_mask_iou[tmp_mask_iou == 0.] = np.inf
tmp_mask_iou[tmp_mask_iou != np.inf] = 0.

# Turns unqualified distances into inf so that they can't be matched together, because inf > dis_thr.
tmp_dis = distances + tmp_mask_iou
forloops_match(tmp_dis, dis_thr)

### Hungarian for second match method with mask_iou > 0

In [ ]:
hungarian_match(tmp_dis, dis_thr)

### Conclusion
Neither Hungarian matching nor forloops work well when matching using only distances in this scenario (same distance).  
And the second match (`mask_iou >0`) can works well in this scenario.

## Test, mask_iou > 0 VS distance+(1-mask_iou) in second match.
In this scenario, we assume that GT and the two Pred have the same Euclidean distance, but one `iou=0.7` and one `iou=0.8`.  
We assume that the Euclidean distance `dis_thr=3`.  
Obviously, we expect the Pred with `index=[0, 1]` to be assigned to this gt, because they have the `iou=0.8`.


In [ ]:
distances = np.array([[1.2, 1.2, 1.8]])
mask_iou = np.array([[0.7, 0.8, 0.]])
dis_thr = 3

### Forloops for mask_iou > 0

In [ ]:
tmp_mask_iou = mask_iou.copy()
tmp_mask_iou[tmp_mask_iou == 0.] = np.inf
tmp_mask_iou[tmp_mask_iou != np.inf] = 0.

# Turns unqualified value into inf so that they can't be matched together.
tmp_dis = distances + tmp_mask_iou
forloops_match(tmp_dis, dis_thr)

### Hungarian for mask_iou > 0

In [ ]:
hungarian_match(tmp_dis, dis_thr)

### Forloops for distance + (1-mask_iou)

In [ ]:
tmp_mask_iou = mask_iou.copy()
tmp_mask_iou[tmp_mask_iou == 0.] = -np.inf
dis_plus_mask = distances + (1-tmp_mask_iou)
forloops_match(dis_plus_mask, dis_thr)

### Hungarian for distance + (1-mask_iou)

In [ ]:
hungarian_match(dis_plus_mask, dis_thr) 

### Conclusion
Obviously, `forloops with distances + (1-mask_iou)` still don't work very well, but `hungarian with distance + (1-mask_iou)` can.


## Test, bbox_iou VS mask_iou

Consider a scenario like the one below, assuming that the aircraft on the left of the figure is gt and the aircraft on the right is pred. Between them `mask_iou=0`, but `bbox_iou!=0`.  
We assume that the `distance=7`,  `bbox_iou=0.1` and `threshold=8`.  

Obviously, we don't want to match them together.  

![](https://songjimagebed.oss-cn-beijing.aliyuncs.com/ce42d1ffec4b48ab59e853ea3f03301.png)

In [ ]:

distances = np.array([[7.0]])
mask_iou = np.array([[0.0]])
bbox_iou = np.array([[0.1]])
dis_thr = 8

### Hungarian for distances + (1-bbox_iou)

In [ ]:
tmp_bbox_iou = bbox_iou.copy()
tmp_bbox_iou[tmp_bbox_iou == 0.] = -np.inf
dis_plus_bbox_iou = distances + (1- tmp_bbox_iou)
hungarian_match(dis_plus_bbox_iou, dis_thr)

### Hungarian for distances + (1-mask_iou)

In [ ]:
tmp_mask_iou = mask_iou.copy()
tmp_mask_iou[tmp_mask_iou == 0.] = -np.inf
dis_plus_mask = distances + (1-tmp_mask_iou)
hungarian_match(dis_plus_mask, dis_thr)

### Conclusion
Obviously, `bbox_iou` can't solve this situation very well, but `mask_iou` can

## Conclusions
Through the above simple experiment, we can find
1. `hungarian_match` is usually more accurate than `forloops_match`.
2. The secondary matching method is more accurate than using only Euclidean distances.
3. `distances + (1-mask_iou)` are more accurate than `mask_iou > 0` in second match method.
4. `mask_iou` is more accurate than `bbox_iou`